In [17]:
!wget --output-document=80_cropped_numbered.zip --no-check-certificate "https://onedrive.live.com/download?cid=DEE6E649D3E6AEAC&resid=DEE6E649D3E6AEAC%2110359&authkey=AHvAYRjICcDlO5o&width=98&height=120&frameborder=0&scrolling=no"

    

--2018-08-31 02:18:30--  https://onedrive.live.com/download?cid=DEE6E649D3E6AEAC&resid=DEE6E649D3E6AEAC%2110359&authkey=AHvAYRjICcDlO5o&width=98&height=120&frameborder=0&scrolling=no
Resolving webproxy (webproxy)... 10.36.41.1
Connecting to webproxy (webproxy)|10.36.41.1|:3128... connected.
Proxy request sent, awaiting response... 302 Found
Location: https://xcqeag.dm.files.1drv.com/y4mWj8zjzCOaPuKVDG5g7PRnKBIHrvYuoQIwszGfUnW9OH7G2Tqdr7k6L9ffHhKsfTu-lNFz4CRaQ0NZuzNjn3SQ5yoMzdIOgFno7q9Uj6fJtrANJjyYgPKL53mnqk1RcdpTNGu1m-PF6WzXHhiRGP68hsFa1czO66Pn3fo48GN6IcM825GZY39s3DsTQ5MsbfzawYeLUNWxW6QthfUyOImug/80_cropped_numbered.zip?download&psid=1 [following]
--2018-08-31 02:18:30--  https://xcqeag.dm.files.1drv.com/y4mWj8zjzCOaPuKVDG5g7PRnKBIHrvYuoQIwszGfUnW9OH7G2Tqdr7k6L9ffHhKsfTu-lNFz4CRaQ0NZuzNjn3SQ5yoMzdIOgFno7q9Uj6fJtrANJjyYgPKL53mnqk1RcdpTNGu1m-PF6WzXHhiRGP68hsFa1czO66Pn3fo48GN6IcM825GZY39s3DsTQ5MsbfzawYeLUNWxW6QthfUyOImug/80_cropped_numbered.zip?download&psid=1
Connecting to webproxy (webp

In [ ]:
!unzip 80.zip

Archive:  80.zip
replace 80/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
!cd speed-limit-signs/0/

80.zip			    __MACOSX	       speed-limit-signs.zip
experiment-inception.ipynb  prediction.ipynb   Untitled1.ipynb
experiment-residual.ipynb   sandbox.ipynb      Untitled.ipynb
hello			    speed-limit-signs  workspace.ipynb


In [11]:
!rm -f 80_cropped_numbered.zip


In [24]:
!ls

100_cropped_numbered	    experiment-residual.ipynb  speed-limit-signs
100_cropped_numbered.zip    hello		       speed-limit-signs.zip
80_cropped_numbered	    __MACOSX		       Untitled1.ipynb
80_cropped_numbered.zip     prediction.ipynb	       Untitled.ipynb
experiment-inception.ipynb  sandbox.ipynb	       workspace.ipynb


In [22]:
!ls

100_cropped_numbered	    experiment-residual.ipynb  speed-limit-signs
100_cropped_numbered.zip    hello		       speed-limit-signs.zip
80_cropped_numbered	    __MACOSX		       Untitled1.ipynb
80_cropped_numbered.zip     prediction.ipynb	       Untitled.ipynb
experiment-inception.ipynb  sandbox.ipynb	       workspace.ipynb


In [1]:
import os
import random
import skimage.data
import skimage.transform
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pdb

import keras as keras
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils.np_utils import to_categorical


import sklearn
from sklearn.model_selection import train_test_split

def load_data(data_dir):
	"""Loads a dta set and returns two lists:
	
	images: a list of Numpy arrays, each representing an image
	labels: a list of numbers that represent the images labels.
		
	"""

	#Get all subdirectories of data_dir. Each represent a label.

	directories = [d for d in os.listdir(data_dir)
				if os.path.isdir(os.path.join(data_dir, d))] #might be an indentation error here
	
	#print directories

	# Loop through the label directories and collect the data in
	# two lists, labels and images

	labels = []
	images = []

	for d in directories:
		label_dir = os.path.join(data_dir, d)
		file_names = [os.path.join(label_dir, f)
						for f in os.listdir(label_dir) if f.endswith(".jpg")]
		# For each label, load it's images and add them to the images list.
		# And add the label number (i.e. directory name) to the labels list.

		#print("%s %s" % (label_dir, file_names))
		#print("\n")	
		for f in file_names:
			images.append(skimage.data.imread(f))
			labels.append(int(d))

	
	print("Unique Labels: {0}\nTotal Images: {1}".format(len(set(labels)), len(images)))
	return images, labels


# Load training and testing datasets

def display_images_and_labels(images, labels):
	"""Display the first image of each label."""
	unique_labels = set(labels)
	plt.figure(figsize=(15, 15))
	i = 1
	for label in unique_labels:
		#Pick the first image for each label
		image = images[labels.index(label)]
		plt.subplot(8, 8, i) #A grid of 8 rows x 8 columns
		plt.axis('off')
		plt.title("Label {0} ({1})".format(label, labels.count(label)))
		i += 1
		_ = plt.imshow(image)
	plt.show()

def display_label_images(images, label):
	"""Display images of a specific label."""
	limit = 24 # show a max of 24 images
	plt.figure(figsize=(15, 5))
	i = 1

	start = labels.index(label)
	end = start + labels.count(label)
	for image in images[start:end][:limit]:
		plt.subplot(3, 8, i) #3 rows, 8 per row
		plt.axis('off')
		i += 1
		plt.imshow(image)
	plt.show()

ROOT_PATH = "./" #
train_data_dir = os.path.join(ROOT_PATH, "alberta_speed_limit_signs") #
#test_data_dir = os.path.join(ROOT_PATH, "datasets/BelgiumTS/Testing")

images, labels = load_data(train_data_dir) #
print len(set(labels)), len(images)

#These two function calls are to display images and labels
#display_images_and_labels(images, labels)
#display_label_images(images, 1)

totalx = 0
totaly = 0
for image in images[:648]: #
	print("Shape: {0}, min: {1}, max: {2}".format(image.shape, image.min(), image.max()))
	#totaly = totaly + image.shape[0]
	#totalx = totalx + image.shape[1]

#print("totaly: {0}\ntotalx: {1}".format((totaly/648), (totalx/648)))

#resize images
images32 = [skimage.transform.resize(image, (64, 64), mode='constant') #
				for image in images]

#This function calls is to display new 32x32 images and labels
#display_images_and_labels(images32, labels)

for image in images32[:5]: #
	print("Shape: {0}, min: {1}, max: {2}".format(image.shape, image.min(), image.max()))

#Minimum Viable Model
# labels_a = np.array(labels)
# images_a = np.array(images32)
# print("labels: ", labels_a.shape, "images: ", images_a.shape)

y = np.array(labels)
X = np.array(images32)
#print y.shape, X.shape

num_categories = 2

y = to_categorical(y, num_categories)

#print y.shape, X.shape


# # input tensor for a 3-channel 64x64 image
inputs = Input(shape=(64, 64, 3))

# # one block of convolutional layers

# # 32 filters with a 4x4 kernel, outputs 64x64x32 tensor
x = Convolution2D(32, 4, 4, border_mode='same', activation='relu')(inputs)
x = Convolution2D(32, 4, 4, border_mode='same', activation='relu')(x)
x = Convolution2D(32, 4, 4, border_mode='same', activation='relu')(x)

# # max pooling with 2x2 window, reducing data to a fourth, reduces risk of overfitting
#x = MaxPooling2D(pool_size=(2, 2))(x)

# # drops 25% of all nodes at training (but not for test/prediction), also reduces risk of overfitting
x = Dropout(0.25)(x)
# # http://cs231n.github.io/neural-networks-2/#reg

# # one more block
x = Convolution2D(64, 4, 4, border_mode='same', activation='relu')(x)
x = Convolution2D(64, 4, 4, border_mode='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

x = Flatten()(x)
# # fully connected, 256 nodes
x = Dense(256, activation='relu')(x)
x = Dropout(0.50)(x)

# # softmax activation, 6 categories
predictions = Dense(2, activation='softmax')(x)

# # this creates a model that includes
# # the Input layer and three Dense layers
model = Model(input=inputs, output=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)
print X_train

# # model.fit(X_train, y_train, nb_epoch=50, batch_size=100)
# # should rather be 50 epochs, but we want it to finish soon and maybe add more cycles if needed
model.fit(X_train, y_train, nb_epoch=20, batch_size=100)



/home/nbuser/anaconda2_501/lib/python2.7/site-packages/pywt/__init__.py:16: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._extensions._pywt import *
/home/nbuser/anaconda2_501/lib/python2.7/site-packages/pywt/_swt.py:1: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._extensions._swt import swt_max_level, swt as _swt, swt_axis as _swt_axis
/home/nbuser/anaconda2_501/lib/python2.7/site-packages/h5py/__init__.py:34: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._conv import register_converters as _register_converters
/home/nbuser/anaconda2_501/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as

Unique Labels: 2
Total Images: 649
2 649
Shape: (39, 34, 3), min: 8, max: 147
Shape: (126, 79, 3), min: 2, max: 243
Shape: (117, 72, 3), min: 103, max: 184
Shape: (77, 25, 3), min: 2, max: 246
Shape: (43, 35, 3), min: 14, max: 135
Shape: (35, 23, 3), min: 0, max: 143
Shape: (58, 49, 3), min: 30, max: 254
Shape: (130, 86, 3), min: 49, max: 178
Shape: (24, 17, 3), min: 37, max: 174
Shape: (53, 33, 3), min: 0, max: 171
Shape: (35, 28, 3), min: 33, max: 150
Shape: (27, 20, 3), min: 21, max: 235
Shape: (31, 26, 3), min: 11, max: 181
Shape: (51, 37, 3), min: 0, max: 151
Shape: (60, 48, 3), min: 21, max: 186
Shape: (197, 123, 3), min: 4, max: 238
Shape: (55, 34, 3), min: 18, max: 230
Shape: (203, 161, 3), min: 11, max: 201
Shape: (37, 28, 3), min: 28, max: 151
Shape: (289, 153, 3), min: 0, max: 182
Shape: (223, 169, 3), min: 1, max: 211
Shape: (53, 42, 3), min: 20, max: 234
Shape: (139, 107, 3), min: 10, max: 190
Shape: (95, 69, 3), min: 12, max: 245
Shape: (109, 82, 3), min: 0, max: 166
Shap

Shape: (122, 89, 3), min: 89, max: 238
Shape: (40, 25, 3), min: 28, max: 183
Shape: (401, 205, 3), min: 0, max: 241
Shape: (86, 64, 3), min: 53, max: 255
Shape: (85, 49, 3), min: 11, max: 208
Shape: (36, 18, 3), min: 72, max: 251
Shape: (143, 43, 3), min: 8, max: 157
Shape: (75, 63, 3), min: 40, max: 165
Shape: (152, 88, 3), min: 29, max: 255
Shape: (297, 186, 3), min: 1, max: 216
Shape: (268, 174, 3), min: 37, max: 255
Shape: (32, 22, 3), min: 31, max: 134
Shape: (119, 78, 3), min: 39, max: 255
Shape: (35, 29, 3), min: 63, max: 166
Shape: (36, 29, 3), min: 23, max: 171
Shape: (91, 62, 3), min: 13, max: 150
Shape: (44, 35, 3), min: 73, max: 240
Shape: (244, 171, 3), min: 4, max: 163
Shape: (86, 60, 3), min: 33, max: 154
Shape: (27, 22, 3), min: 34, max: 170
Shape: (56, 41, 3), min: 4, max: 145
Shape: (213, 151, 3), min: 0, max: 141
Shape: (298, 204, 3), min: 6, max: 252
Shape: (119, 87, 3), min: 1, max: 153
Shape: (83, 71, 3), min: 35, max: 143
Shape: (118, 94, 3), min: 4, max: 141
Sha

Shape: (64, 64, 3), min: 0.0313725490196, max: 0.572861615349
Shape: (64, 64, 3), min: 0.017027611826, max: 0.94929486443
Shape: (64, 64, 3), min: 0.408162913603, max: 0.720586320466
Shape: (64, 64, 3), min: 0.0194869695925, max: 0.954557770374
Shape: (64, 64, 3), min: 0.0549019607843, max: 0.529361261106


/home/nbuser/anaconda2_501/lib/python2.7/site-packages/ipykernel/__main__.py:144: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (4, 4), padding="same", activation="relu")`
/home/nbuser/anaconda2_501/lib/python2.7/site-packages/ipykernel/__main__.py:145: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (4, 4), padding="same", activation="relu")`
/home/nbuser/anaconda2_501/lib/python2.7/site-packages/ipykernel/__main__.py:146: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (4, 4), padding="same", activation="relu")`
/home/nbuser/anaconda2_501/lib/python2.7/site-packages/ipykernel/__main__.py:156: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (4, 4), padding="same", activation="relu")`
/home/nbuser/anaconda2_501/lib/python2.7/site-packages/ipykernel/__main__.py:157: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (4, 4), padding="same", activation="relu")`
/home/nbuser/an

[[[[0.15283251 0.17127781 0.15019747]
   [0.20962943 0.23300566 0.20751618]
   [0.23472757 0.25810379 0.23454494]
   ...
   [0.17091256 0.20430717 0.17388677]
   [0.14195317 0.17534778 0.14117647]
   [0.14117647 0.14117647 0.14117647]]

  [[0.21380113 0.23546167 0.21070676]
   [0.29127269 0.31872367 0.28799235]
   [0.32326086 0.35071184 0.32099011]
   ...
   [0.23503274 0.27424843 0.23852539]
   [0.19599514 0.23521082 0.19495347]
   [0.14117647 0.1660319  0.14117647]]

  [[0.25013571 0.27259402 0.24385029]
   [0.33971665 0.36817866 0.33249344]
   [0.37547751 0.40393952 0.3694233 ]
   ...
   [0.27242575 0.30866364 0.27389634]
   [0.22642727 0.26324966 0.22336354]
   [0.1556126  0.1849607  0.15092271]]

  ...

  [[0.27150616 0.28927457 0.18566176]
   [0.3621089  0.38462728 0.253316  ]
   [0.39048378 0.41300216 0.28169089]
   ...
   [0.4096627  0.44007951 0.36027545]
   [0.38355497 0.41455629 0.32963341]
   [0.28956466 0.31431957 0.24474475]]

  [[0.26403617 0.28260235 0.17739401]
   [0.3

Epoch 1/20
519/519 [==============================] - 170s - loss: 4.9517 - acc: 0.5742   
Epoch 2/20
519/519 [==============================] - 172s - loss: 5.7764 - acc: 0.6416   
Epoch 3/20
519/519 [==============================] - 168s - loss: 5.7764 - acc: 0.6416   
Epoch 4/20
519/519 [==============================] - 168s - loss: 5.7764 - acc: 0.6416   
Epoch 5/20


KeyboardInterrupt: 

In [7]:
!git init

Reinitialized existing Git repository in /home/nbuser/library/.git/
